In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!rm -rf ~/.cache/huggingface/hub/models--FacebookAI--xlm-roberta-large


In [ ]:
pip install transformers==4.41.1


In [ ]:
# -------------------------------
# 0. Environment Setup (Run Once)
# -------------------------------
!pip uninstall -y peft
!pip install transformers==4.41.1 datasets --upgrade

# Restart runtime after this cell before running the rest


In [ ]:
pip install transformers==4.41.1


In [ ]:
pip install --upgrade transformers peft


In [10]:
!pip install -q transformers datasets scikit-learn torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
import torch
torch.cuda.empty_cache()


In [19]:
# =============================
# NLP Project: Emotion Classification (Kaggle-ready)
# GPU-safe version
# =============================

!pip install -q transformers datasets scikit-learn torch

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pickle

# -----------------------------
# 1. Load Dataset
# -----------------------------
train_df = pd.read_csv("/kaggle/input/nlpdataset1/train.csv")[['Sentence', 'Emotion']]
val_df = pd.read_csv("/kaggle/input/nlpdataset1/val.csv")[['Sentence', 'Emotion']]
test_df = pd.read_csv("/kaggle/input/nlpdataset1/test.csv")[['Sentence', 'Emotion']]

# Encode Emotion labels
le = LabelEncoder()
train_df['Emotion'] = le.fit_transform(train_df['Emotion'])
val_df['Emotion'] = le.transform(val_df['Emotion'])
test_df['Emotion'] = le.transform(test_df['Emotion'])
num_labels = len(le.classes_)

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# -----------------------------
# 2. Model Selection (GPU-safe)
# -----------------------------
model_names = [
    "xlm-roberta-base",  # smaller than large
    "distilbert-base-multilingual-cased",
    "sentence-transformers/indic-bert",
    "bert-base-multilingual-cased"
]

# -----------------------------
# 3. Tokenization Function
# -----------------------------
def tokenize_function(examples, tokenizer):
    return tokenizer(examples["Sentence"], padding="max_length", truncation=True, max_length=128)

# -----------------------------
# 4. Train Function
# -----------------------------
def train_model(model_name, batch_size=4, fp16=True):
    print(f"\n===== Training {model_name} =====")
    
    # Clear GPU cache before starting
    torch.cuda.empty_cache()
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    
    # Tokenize datasets
    tokenized_train = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_val = val_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_test = test_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    
    # Rename label column for Trainer
    tokenized_train = tokenized_train.rename_column("Emotion", "labels")
    tokenized_val = tokenized_val.rename_column("Emotion", "labels")
    tokenized_test = tokenized_test.rename_column("Emotion", "labels")
    
    # Set format for PyTorch
    tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    tokenized_val.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./results_{model_name.replace('/', '_')}",
        num_train_epochs=3,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=2e-5,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="no",
        logging_strategy="steps",
        logging_steps=50,
        report_to="none",
        load_best_model_at_end=False,
        fp16=fp16  # mixed precision
    )
    
    # Metrics function
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, predictions)
        return {"accuracy": acc}
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics
    )
    
    # Train
    trainer.train()
    
    # Predict on test set
    preds_output = trainer.predict(tokenized_test)
    preds = np.argmax(preds_output.predictions, axis=-1)
    
    # Confusion matrix & classification report
    cm = confusion_matrix(test_df['Emotion'], preds)
    report = classification_report(test_df['Emotion'], preds, target_names=le.classes_, output_dict=True)
    acc = accuracy_score(test_df['Emotion'], preds)
    
    # Delete model to free memory
    del model
    torch.cuda.empty_cache()
    
    print(f"✅ {model_name} Accuracy: {acc:.4f}")
    print(f"Confusion Matrix:\n{cm}\n")
    
    return {
        "model_name": model_name,
        "tokenizer": tokenizer,
        "preds": preds,
        "cm": cm,
        "report": report,
        "accuracy": acc
    }

# -----------------------------
# 5. Train All Models
# -----------------------------
model_results = []
for model_name in model_names:
    res = train_model(model_name, batch_size=4, fp16=True)
    model_results.append(res)

# -----------------------------
# 6. Stacking Ensemble (Majority Voting)
# -----------------------------
all_preds = np.array([res['preds'] for res in model_results])
stacked_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=all_preds)
stacked_cm = confusion_matrix(test_df['Emotion'], stacked_preds)
stacked_report = classification_report(test_df['Emotion'], stacked_preds, target_names=le.classes_, output_dict=True)
stacked_acc = accuracy_score(test_df['Emotion'], stacked_preds)

stacked_result = {
    "model_name": "Stacked_Ensemble",
    "preds": stacked_preds,
    "cm": stacked_cm,
    "report": stacked_report,
    "accuracy": stacked_acc
}

print(f"✅ Stacked Ensemble Accuracy: {stacked_acc:.4f}")
print(f"Confusion Matrix:\n{stacked_cm}\n")

# -----------------------------
# 7. Save Results in Pickle
# -----------------------------
all_results = {
    "label_encoder": le,
    "individual_model_results": model_results,
    "stacked_result": stacked_result
}

with open("emotion_model_results.pkl", "wb") as f:
    pickle.dump(all_results, f)

print("✅ Training complete! Results saved to emotion_model_results.pkl")



===== Training xlm-roberta-base =====


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.751100,0.710760,0.728730
2,0.665900,0.663926,0.752569
3,0.448200,0.702256,0.761200


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ distilbert-base-multilingual-cased Accuracy: 0.7457
Confusion Matrix:
[[   8    0    2   11   17]
 [   2    3    0    6    1]
 [   4    0  219  153   24]
 [   2    0   99 1250  121]
 [   7    2   21  147  335]]


===== Training sentence-transformers/indic-bert =====


OSError: sentence-transformers/indic-bert is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

xlm-roberta-classification

In [29]:
import numpy as np
from sklearn.metrics import classification_report

# Confusion matrix
cm = np.array([
    [16, 0, 1, 9, 12],
    [4, 0, 0, 5, 3],
    [2, 0, 278, 114, 6],
    [4, 0, 106, 1245, 117],
    [11, 0, 20, 108, 373]
])

# Class labels
labels = ["angry", "fear", "happy", "no", "sad"]

# Derive predictions and true labels from confusion matrix
y_true = []
y_pred = []
for i in range(len(cm)):
    for j in range(len(cm)):
        y_true += [labels[i]] * cm[i, j]
        y_pred += [labels[j]] * cm[i, j]

# Generate classification report
report = classification_report(y_true, y_pred, target_names=labels, digits=3)
print("Classification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

       angry      0.432     0.421     0.427        38
        fear      0.000     0.000     0.000        12
       happy      0.686     0.695     0.691       400
          no      0.841     0.846     0.843      1472
         sad      0.730     0.729     0.729       512

    accuracy                          0.786      2434
   macro avg      0.538     0.538     0.538      2434
weighted avg      0.781     0.786     0.784      2434



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
!pip install -q transformers datasets scikit-learn torch

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pickle

# -----------------------------
# 1. Load Dataset
# -----------------------------
train_df = pd.read_csv("/kaggle/input/nlpdataset1/train.csv")[['Sentence', 'Emotion']]
val_df = pd.read_csv("/kaggle/input/nlpdataset1/val.csv")[['Sentence', 'Emotion']]
test_df = pd.read_csv("/kaggle/input/nlpdataset1/test.csv")[['Sentence', 'Emotion']]

# Encode Emotion labels
le = LabelEncoder()
train_df['Emotion'] = le.fit_transform(train_df['Emotion'])
val_df['Emotion'] = le.transform(val_df['Emotion'])
test_df['Emotion'] = le.transform(test_df['Emotion'])
num_labels = len(le.classes_)

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# -----------------------------
# 2. Tokenization Function
# -----------------------------
def tokenize_function(examples, tokenizer):
    return tokenizer(examples["Sentence"], padding="max_length", truncation=True, max_length=128)

# -----------------------------
# 3. Train Function for Third Model
# -----------------------------
def train_third_model(model_name="ai4bharat/IndicBERTv2-SS"):
    print(f"\n===== Training {model_name} =====")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    # ✅ Handle multi-GPU safely
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel...")
        model = torch.nn.DataParallel(model)

    model.to(device)

    # Tokenize datasets
    tokenized_train = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_val = val_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_test = test_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    # Rename label column for Trainer
    tokenized_train = tokenized_train.rename_column("Emotion", "labels")
    tokenized_val = tokenized_val.rename_column("Emotion", "labels")
    tokenized_test = tokenized_test.rename_column("Emotion", "labels")

    # Set format for PyTorch
    tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    tokenized_val.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

    # -----------------------------
    # Training arguments (FIXED)
    # -----------------------------
    training_args = TrainingArguments(
        output_dir=f"./results_{model_name.replace('/', '_')}",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="no",
        logging_strategy="steps",
        logging_steps=50,
        report_to="none",
        load_best_model_at_end=False,
        remove_unused_columns=False  # ✅ FIX FOR THE ERROR
    )

    # Metrics function
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, predictions)
        return {"accuracy": acc}

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics
    )

    # Train
    trainer.train()

    # Predict on test set
    preds_output = trainer.predict(tokenized_test)
    preds = np.argmax(preds_output.predictions, axis=-1)

    # Confusion matrix & classification report
    cm = confusion_matrix(test_df['Emotion'], preds)
    report = classification_report(test_df['Emotion'], preds, target_names=le.classes_, output_dict=True)
    acc = accuracy_score(test_df['Emotion'], preds)

    # Save results
    results = {
        "model_name": model_name,
        "tokenizer": tokenizer,
        "model": model,
        "preds": preds,
        "cm": cm,
        "report": report,
        "accuracy": acc
    }

    with open("third_model_results.pkl", "wb") as f:
        pickle.dump(results, f)

    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nAccuracy: {acc}")
    print("\nClassification Report:")
    print(pd.DataFrame(report).transpose())

    return results

# -----------------------------
# 4. Run Training
# -----------------------------
third_model_results = train_third_model()



===== Training ai4bharat/IndicBERTv2-SS =====


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.956900,0.970157,0.619400
2,0.911700,0.912590,0.638718
3,0.882200,0.879578,0.648582


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Confusion Matrix:
[[   0    0    1   22   15]
 [   0    0    0    9    3]
 [   0    0   71  283   46]
 [   0    0   51 1300  121]
 [   0    0   17  276  219]]

Accuracy: 0.6532456861133936

Classification Report:
              precision    recall  f1-score      support
angry          0.000000  0.000000  0.000000    38.000000
fear           0.000000  0.000000  0.000000    12.000000
happy          0.507143  0.177500  0.262963   400.000000
no             0.687831  0.883152  0.773349  1472.000000
sad            0.542079  0.427734  0.478166   512.000000
accuracy       0.653246  0.653246  0.653246     0.653246
macro avg      0.347411  0.297677  0.302896  2434.000000
weighted avg   0.613348  0.653246  0.611494  2434.000000


bert-base

In [28]:


import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pickle

# -----------------------------
# 1. Load Dataset
# -----------------------------
train_df = pd.read_csv("/kaggle/input/nlpdataset1/train.csv")[['Sentence', 'Emotion']]
val_df = pd.read_csv("/kaggle/input/nlpdataset1/val.csv")[['Sentence', 'Emotion']]
test_df = pd.read_csv("/kaggle/input/nlpdataset1/test.csv")[['Sentence', 'Emotion']]

# Encode Emotion labels
le = LabelEncoder()
train_df['Emotion'] = le.fit_transform(train_df['Emotion'])
val_df['Emotion'] = le.transform(val_df['Emotion'])
test_df['Emotion'] = le.transform(test_df['Emotion'])
num_labels = len(le.classes_)

# Convert to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# -----------------------------
# 2. Tokenization Function
# -----------------------------
def tokenize_function(examples, tokenizer):
    return tokenizer(examples["Sentence"], padding="max_length", truncation=True, max_length=128)

# -----------------------------
# 3. Train Function for Fourth Model
# -----------------------------
def train_fourth_model(model_name="bert-base-multilingual-cased"):
    print(f"\n===== Training {model_name} =====")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    # Tokenize datasets
    tokenized_train = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_val = val_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    tokenized_test = test_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    # Rename label column for Trainer
    tokenized_train = tokenized_train.rename_column("Emotion", "labels")
    tokenized_val = tokenized_val.rename_column("Emotion", "labels")
    tokenized_test = tokenized_test.rename_column("Emotion", "labels")

    # Set format for PyTorch
    tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    tokenized_val.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./results_{model_name.replace('/', '_')}",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="no",
        logging_strategy="steps",
        logging_steps=50,
        report_to="none",
        load_best_model_at_end=False,
        fp16=True  # enable mixed precision for speed
    )

    # Metrics function
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, predictions)
        return {"accuracy": acc}

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics
    )

    # Train
    trainer.train()

    # Predict on test set
    preds_output = trainer.predict(tokenized_test)
    preds = np.argmax(preds_output.predictions, axis=-1)

    # Confusion matrix & classification report
    cm = confusion_matrix(test_df['Emotion'], preds)
    report = classification_report(test_df['Emotion'], preds, target_names=le.classes_, output_dict=True)
    acc = accuracy_score(test_df['Emotion'], preds)

    # Save to pickle
    results = {
        "model_name": model_name,
        "tokenizer": tokenizer,
        "model": model,
        "preds": preds,
        "cm": cm,
        "report": report,
        "accuracy": acc
    }

    with open("fourth_model_results.pkl", "wb") as f:
        pickle.dump(results, f)

    # Print outputs
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nAccuracy: {acc}")
    print("\nClassification Report:")
    print(pd.DataFrame(report).transpose())

    return results

# -----------------------------
# 4. Run Training
# -----------------------------
bertbase = train_fourth_model()



===== Training bert-base-multilingual-cased =====


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/19464 [00:00<?, ? examples/s]

Map:   0%|          | 0/2433 [00:00<?, ? examples/s]

Map:   0%|          | 0/2434 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.711000,0.676383,0.747637
2,0.609900,0.612836,0.766543
3,0.490500,0.620043,0.772298


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Confusion Matrix:
[[   8    0    3    9   18]
 [   2    3    0    4    3]
 [   3    0  233  143   21]
 [   0    0   99 1254  119]
 [   9    3   34  121  345]]

Accuracy: 0.7571898110106821

Classification Report:
              precision    recall  f1-score     support
angry          0.363636  0.210526  0.266667    38.00000
fear           0.500000  0.250000  0.333333    12.00000
happy          0.631436  0.582500  0.605982   400.00000
no             0.819073  0.851902  0.835165  1472.00000
sad            0.681818  0.673828  0.677800   512.00000
accuracy       0.757190  0.757190  0.757190     0.75719
macro avg      0.599193  0.513751  0.543789  2434.00000
weighted avg   0.750681  0.757190  0.753049  2434.00000
